In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset.head()

## Preprocesamiento de los datos


Comprobamos si el dataset contiene valores desconocidos

In [ ]:
dataset.isna().sum()

Como se puede observar, el conjunto de datos no presenta ningún valor desconocido.

Si analizamos las columnas del dataset, podemos observar que los features: *sex, smoker, region* son categóricos. Por ello, convertimos los datos categóricos a numéricos.

In [ ]:
dataset['region'].unique()

In [ ]:
dataset['sex'].replace(['female', 'male'], [0,1], inplace=True)
dataset['smoker'].replace(['yes', 'no'], [1,0], inplace=True)
dataset['region'].replace(['southwest', 'southeast', 'northwest', 'northeast'], [0,1,2,3], inplace=True)

dataset.head()

### Train & Test datasets


Tenemos que usar el 80% de los datos para el conjunto de entrenamiento y el 20% restante para el conjunto de prueba.

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
train_dataset.shape

In [ ]:
test_dataset.shape

### Inspeccionamos los datos

In [ ]:
import seaborn as sns
sns.pairplot(train_dataset[['age','bmi','children','expenses']], diag_kind='kde')

También podemos revisar las estadísticas generales

In [ ]:
train_stats = train_dataset.describe()
train_stats.pop('expenses')
train_stats = train_stats.transpose()
train_stats

### Separamos los features del label

Separamos la etiqueta que servirá para entrenar el modelo

In [ ]:
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

### Normalizamos los datos

Aunque el modelo pudiera converger sin normalizar las caracteristicas, dificulta el entrenamiento y hace que el modelo resultante dependa de la elección de las unidades utlizadas en la entrada

In [ ]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']


normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

## Creación del modelo

In [ ]:
def build_model():
  model = keras.Sequential([
      layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]), #1st hidden layer
      layers.Dense(64, activation='relu'), # 2nd hidden layer
      layers.Dense(1) # output layer
  ])

  model.compile(
      optimizer = keras.optimizers.RMSprop(0.001),
      loss = 'mse', # Mean Square Error
      metrics = ['mae', 'mse'] # Mean Absolute Error , Mean Square Error
  )

  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

## Entrenamiento del modelo

In [ ]:
model = build_model()

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10) # patience es la cantidad de epochs que chequea para compronar si hay mejoras

history = model.fit(normed_train_data, train_labels, epochs=1000, validation_split=0.2, verbose=0)

## Evaluando el modelo

In [ ]:
test_dataset = normed_test_data

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
